# Big Data y Machine Learning para clasificación de galaxias

- [Docker - Spark](#all-spark)
    - [Jupyter Docker Stacks](#jds)
- [Dataset de imágenes galácticas](#imagenes)
    - [Script de imágenes](#script)

    
<div id='xx' />

<div id='all-spark' />

## Docker - Spark

En este punto del proyecto vamos a hacer un resumen de lo que hemos hecho hasta el momento:

- Configuración del entorno de trabajo Cloudera.
- Descarga da datos: dataset de parámetros y GalaxyZoo.
- Almacenamiento en HDFS.
- Almacenamiento en HIVE.
- Consultas a través de HUE y ejemplo de análisis usando SPARK.

El trabajo que nos queda por llevar a cabo sería el siguiente:

- Desarrollo de script para descarga y procesamiento de imágenes.
- Desarrollo de algoritmos de clasificación (machine learning).

Sin embargo, nos hemos encontrado con una dificultad técnica durante la ejecución de éstos dos últimos apartados y que tiene relación con los requisitos de memoria RAM de nuestro entorno Cloudera. Hasta el momento nuestro contenedor Cloudera consume aproximadamente unos 10GB de memoria RAM sobre una máquina de 16GB de RAM. En el momento en el que intentamos cargar en memoria los datos de las imágenes y usar este dataset para entrenar nuestros modelos de machine learning nos encontramos con un error JAVA que nos indica que no tenemos suficiente memoria alojada en nuestro clúster (recordemos que nuestro entorno es standalone) para concluir con los jobs de SPARK. Dada la imposibilidad de llevar a cabo el entrenamiento de un modelo en Cloudera y debido a las insuficiencias técnicas descritas, hemos considerado la posibilidad de trasladar nuestro posterior trabajo a otro entorno que nos permita concluir con nuestro proyecto. El entorno elegido es un contenedor basado en las imágenes que ofece Jupyter Labs: [Jupyter Docker Stacks](https://jupyter-docker-stacks.readthedocs.io/en/latest/index.html)

<div id='jds' />

### Jupyter Docker Stacks

Jupyter Docker Stacks consiste en una serie de imágenes Docker ya preparadas con un conjunto de aplicaciones y herramientas de computación. En especial resultan muy útiles durante el día a día de todo data scientist ya que contienen el stack típico de herramientas que suele necesitar. Por lo tanto, para el desarrollo de esta segunda parte de nuestro proyecto, y con la idea de seguir trabajando con SPARK, hemos decidido descargar la imagen [jupyter/all-spark-notebook](https://hub.docker.com/r/jupyter/all-spark-notebook/). Con las siguientes instrucciones tenemos nuestro entorno ya listo para trabajar:

`
$ docker pull jupyter/all-spark-notebook
$ docker run -d --name spark -p 8888:8888 -v /home/<user>/cloudera:/home/jovyan/work jupyter/all-spark-notebook:latest
`

Así tenemos un contenedor llamado *spark* con nuestros *notebooks jupyter* accesibles en el puerto 8888 y con un volumen persistente que será nuestro directorio de trabajo. Para acceder a nuestro laboratorio tecleamos en consola y pegamos en nuestro navegador la url devuelta:

`
$ docker exec -t spark jupyter-notebook list
`

<div id='imagenes' />

## Dataset de imágenes galácticas

Tal como hicimos en el apartado de *Descarga de datos: Dataset GalaxyZoo*, procedemos a la descarga en nuestro nuevo entorno.

In [ ]:
!wget http://galaxy-zoo-1.s3.amazonaws.com/GalaxyZoo1_DR_table2.csv.zip
!unzip GalaxyZoo1_DR_table2.csv.zip

En el siguiente [enlace](https://data.galaxyzoo.org/) tenemos toda la información sobre el dataset descargado, así como algunos artículos publicados sobre el proyecto *GalaxyZoo* y *SDSS*. 

In [ ]:
!mkdir data
!mv GalaxyZoo1_DR_table2.csv data
!cat data/GalaxyZoo1_DR_table2.csv | head -n3

Del conjunto de variables con las que cuenta nuestro dataset nos interesan solo las siguientes (y cuyo significado ya hemos explicado con anterioridad): OBJID, RA, DEC, SPIRAL, ELLIPTICAL, UNCERTAIN. Serán estas variables las que usaremos para descargar las imágenes y construir nuestro dataset de entrenamiento.

<div id='script' />

### Script de imágenes

Para descargar las imágenes recurriremos a la siguiente [web](https://skyserver.sdss.org/dr14/en/tools/chart/listinfo.aspx) del proyecto *SDSS*. Seleccionando el nombre y los parámetros de nuestra imagen podemos visualizar un conjunto de imágenes del *DataRelease 14*. Nosotros automatizaremos el proceso de descarga y procesado mediante el siguiente script que ya hemos explicado anteriormente. En resumen:

- Descargamos 10000 imágenes de tamaño `64x64` en base al dataset de *GalaxyZoo*.


- Cargamos las imágenes y las transformamos a escala de grises.


- Construimos un vector con los píxeles de dichas imágenes.


- Añadimos el `drobjid` y el `target` a dicho vector. El `target` lo construimos en base a la función `our_target()`, a la cual pasamos los valores consecutivos `uncertain+spiral+elliptical` del dataset *GalaxyZoo* como string y que en función de su valor (en binario) hacemos corresponder la clase/tipo de galaxia.


    - uncertain+spiral+elliptical = 100 -> target = 0 -> clase = incierto
    - uncertain+spiral+elliptical = 001 -> target = 1 -> clase = elíptica
    - uncertain+spiral+elliptical = 010 -> target = 2 -> clase = espiral
    
    
- Construimos un dataset que contiene las siguientes variables:

    - drobjid: identificador del objeto.
    - target: clase del objeto.
    - F0 a F4095: correspondiente al vector de 4095 atributos de la imagen.

In [ ]:
!mkdir image-galaxyzoo

In [ ]:
from time import time
import numpy as np
import random
import requests
import csv
import cv2

# Función para seleccionar la clase: 0 incierto, 1 elíptica, 2 espiral
def our_target(string):
    target = int(str(string), 2)
    if target == 4:
        target -= 4
    return target

# Contador de tiempo
start = time()

# Tamaño de las imágenes
width, height = str(60), str(60)

# Semilla
random.seed(30)

# Data
name = !cat data/GalaxyZoo1_DR_table2.csv | cut -f "1" -d "," | awk 'NR > 1'
ra = !cat data/GalaxyZoo1_DR_table2.csv | cut -f "2" -d "," | awk 'NR > 1'
dec = !cat data/GalaxyZoo1_DR_table2.csv | cut -f "3" -d "," | awk 'NR > 1'
spiral = !cat data/GalaxyZoo1_DR_table2.csv | cut -f "14" -d "," | awk 'NR > 1'
elliptical = !cat data/GalaxyZoo1_DR_table2.csv | cut -f "15" -d "," | awk 'NR > 1'
uncertain = !cat data/GalaxyZoo1_DR_table2.csv | cut -f "16" -d "," | awk 'NR > 1'

# Seleccionaremios un conjunto de datos balanceados: 4000 spiral, 4000 elliptical, 2000 uncertain (40+40+20)
# Índices que contienen "1"
spiral_index = [i for i, e in enumerate(spiral) if e == "1"]
elliptical_index = [i for i, e in enumerate(elliptical) if e == "1"]
uncertain_index = [i for i, e in enumerate(uncertain) if e == "1"]

# Selección aleatoria 40+40+20
spiral_choices = random.choices(spiral_index, k=4000)
elliptical_choices = random.choices(elliptical_index, k=4000)
uncertain_choices = random.choices(uncertain_index, k=2000)

# Construimos vector que contiene los índices de las imágenes que cumplen la condición 40+40+20
index = [spiral_choices, elliptical_choices, uncertain_choices]
index = [p for q in index for p in q]

print("Número de filas dataset GalaxyZoo: ", len(name))

# Contador de imágenes
j = 1

for i in index:
    # Descarga de imagen
    url_image = "http://skyserver.sdss.org/dr14/SkyServerWS/ImgCutout/getjpeg?TaskName=Skyserver.Chart.List&ra="\
                +ra[i]+"&dec="+dec[i]+"&scale=0.4&width="+width+"&height="+height+"&opt="
    
    local_name = "image-galaxyzoo/" + name[i] + ".png"
    image = requests.get(url_image).content
    
    with open(local_name, 'wb') as png:
        png.write(image)
    
    # Construimos target
    target = our_target(uncertain[i]+spiral[i]+elliptical[i])
    # Carga imagen a gris   
    image_gray = cv2.imread(local_name, 0)
    # Procesamiento imagen a vector
    image_vector = np.reshape(image_gray, -1) / 255
    # Construimos vector id, target, F0...
    image_vector = image_vector.tolist()
    image_vector.insert(0, target)
    image_vector.insert(0, name[i])
    
    print(j, "imágenes procesadas de", len(name)-1)
    
    # Construcción dataset
    with open('data/T_F_DR14_ZooSpec_10000.csv', mode='a+', newline='') as csv_file:
        writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        if j == 1:
            header = [str("F") + str(x) for x in range(0, image_gray.size)]
            header.insert(0, "target")
            header.insert(0, "dr7objid")
            writer.writerow(header)
            
        writer.writerow(image_vector)
    j += 1
    
print(f"Tiempo de ejecución: {(time() - start) / 60} (min)")

Como resultado de la descarga, tenemos un directorio `data` que contiene el dataset de *GalaxyZoo* y nuestro dataset de features *T_F_DR14_ZooSpec_10000.csv*. Además, en el directorio `image-galaxyzoo` tenemos 10000 imágenes de galaxias en formato `.png`: 4000 imágenes corresponden a la clase espiral, otras 4000 a la clase elíptica mientras que el resto (2000) corresponde a imágenes no clasificadas (incierto).

Aunque la descarga la hemos hecho en nuestro nuevo entorno SPARK, el procedimiento sobre el entorno Cloudera sería exactamente el mismo con el único detalle de que nuestros datos quedarían guardado en HDFS.